In [1]:
import os
import time
import pandas as pd
import numpy as np
from geojson import dump

In [2]:
OXF_MSOA_df =  pd.read_excel('data/msoa_data.xlsx', usecols=["msoa11cd"]) # import file with OXF MSOA codes

In [3]:
OXF_MSOA_df.columns = ['areakey'] # rename column "msoa11cd" to "areakey"
OXF_MSOA_list = OXF_MSOA_df['areakey'].tolist()

In [4]:
del OXF_MSOA_list[-1]

In [5]:
zonecodes_EWS = pd.read_csv('data/EWS_ZoneCodes.csv')

In [6]:
zonecodes_EWS.set_index('areakey')
zonecodes_EWS_list = zonecodes_EWS['areakey'].tolist()

In [7]:
import numpy as np
import pickle
import struct
import csv

def loadQUANTMatrix(filename):
    with open(filename,'rb') as f:
        (m,) = struct.unpack('i', f.read(4))
        (n,) = struct.unpack('i', f.read(4))
        print("loadQUANTMatrix::m=",m,"n=",n)
        matrix = np.arange(m*n,dtype=np.float64).reshape(m, n) #and hopefully m===n, but I'm not relying on it
        for i in range(0,m):
            data = struct.unpack('{0}f'.format(n), f.read(4*n)) #read a row back from the stream - data=list of floats
            for j in range(0,n):
                matrix[i,j] = data[j]
    return matrix

In [8]:
cij_road_EWS = loadQUANTMatrix('data/dis_roads_min.bin')

loadQUANTMatrix::m= 8436 n= 8436


In [9]:
cij_road_EWS_df = pd.DataFrame(cij_road_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list)

In [10]:
cij_road_OXF_df = cij_road_EWS_df[OXF_MSOA_list]

In [11]:
cij_road_OXF_df = cij_road_OXF_df.loc[OXF_MSOA_list]

In [12]:
cij_road_OXF = cij_road_OXF_df.to_numpy() 

In [13]:
cij_road_OXF[cij_road_OXF < 1] = 1 

In [14]:
def saveMatrix(matrix,filename):
    with open(filename,'wb') as f:
        pickle.dump(matrix,f)

In [15]:
saveMatrix(cij_road_OXF, os.path.join('data','Cij_road_min_OXF.bin'))

In [16]:
np.savetxt(os.path.join('data', "cij_road_OXF.csv"), cij_road_OXF, delimiter=",")

In [17]:
cij_bus_EWS = loadQUANTMatrix('data/dis_bus_min.bin')
cij_bus_EWS_df = pd.DataFrame(cij_bus_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list) # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
cij_bus_OXF_df = cij_bus_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
cij_bus_OXF_df = cij_bus_OXF_df.loc[OXF_MSOA_list]  # Filter rows
cij_bus_OXF = cij_bus_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)
cij_bus_OXF[cij_bus_OXF < 1] = 1  # lower limit of 1 minute links
saveMatrix(cij_bus_OXF, os.path.join('data','Cij_bus_min_OXF.bin'))
        # save as csv file
np.savetxt(os.path.join('data', "cij_bus_OXF.csv"), cij_bus_OXF, delimiter=",")

loadQUANTMatrix::m= 8436 n= 8436


In [18]:
cij_rail_EWS = loadQUANTMatrix('data/dis_gbrail_min.bin')
cij_rail_EWS_df = pd.DataFrame(cij_rail_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list) # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
cij_rail_OXF_df = cij_rail_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
cij_rail_OXF_df = cij_rail_OXF_df.loc[OXF_MSOA_list]  # Filter rows
cij_rail_OXF = cij_rail_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)
cij_rail_OXF[cij_rail_OXF < 1] = 1  # lower limit of 1 minute links
saveMatrix(cij_rail_OXF, os.path.join('data', 'Cij_gbrail_min_OXF.bin'))
        # save as csv file
np.savetxt(os.path.join('data', "cij_rail_OXF.csv"), cij_rail_OXF, delimiter=",")

loadQUANTMatrix::m= 8436 n= 8436


In [19]:
SObs_road_EWS = loadQUANTMatrix('data/SObs_1.bin')
SObs_road_EWS_df = pd.DataFrame(SObs_road_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list) # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
SObs_road_OXF_df = SObs_road_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
SObs_road_OXF_df = SObs_road_OXF_df.loc[OXF_MSOA_list]  # Filter rows
SObs_road_OXF = SObs_road_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)
saveMatrix(SObs_road_OXF, os.path.join('data', 'SObs_1_OXF.bin'))

loadQUANTMatrix::m= 8436 n= 8436


In [20]:
SObs_bus_EWS = loadQUANTMatrix('data/SObs_2.bin')
SObs_bus_EWS_df = pd.DataFrame(SObs_bus_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list)  # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
SObs_bus_OXF_df = SObs_bus_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
SObs_bus_OXF_df = SObs_bus_OXF_df.loc[OXF_MSOA_list]  # Filter rows
SObs_bus_OXF = SObs_bus_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)
saveMatrix(SObs_bus_OXF, os.path.join('data','SObs_2_OXF.bin'))

loadQUANTMatrix::m= 8436 n= 8436


In [21]:
SObs_rail_EWS = loadQUANTMatrix('data/SObs_3.bin')
SObs_rail_EWS_df = pd.DataFrame(SObs_rail_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list)  # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
SObs_rail_OXF_df = SObs_rail_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
SObs_rail_OXF_df = SObs_rail_OXF_df.loc[OXF_MSOA_list]  # Filter rows
SObs_rail_OXF = SObs_rail_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)
saveMatrix(SObs_rail_OXF, os.path.join('data','SObs_3_OXF.bin'))

loadQUANTMatrix::m= 8436 n= 8436


In [22]:
dfEW = pd.read_csv('data/QS103EW_MSOA.csv')
dfEW['count_allpeople'] = dfEW['Age: All categories: Age; measures: Value'] # you could just rename the col, not copy
dfEW2 = pd.DataFrame({'geography code': dfEW['geography code'], 'count_allpeople': dfEW['count_allpeople']})

In [23]:
dfS = pd.read_csv('data/QS103SC_DZ2001.csv') # join on "Unnamed: 0", it's blank! This is the datazone code field
dfS.set_index('Unnamed: 0')
dfSLookup = pd.read_csv('data/DZ2001Lookup.csv') # join on ZONECODE, which is the datazone code
dfS = dfS.join(other=dfSLookup.set_index('ZONECODE'),on='Unnamed: 0')
dfS['count_allpeople'] = dfS['All people']
dfS2 = dfS.groupby(['IZ_CODE']).agg({'count_allpeople': "sum"})
dfS3 = pd.DataFrame({'msoaiz': dfS2.index, 'count_allpeople': dfS2['count_allpeople'] })

In [24]:
dfEW2.reset_index()
dfS3.reset_index()
dfEW2.columns=['msoaiz','count_allpeople']
dfEWS = pd.concat([dfEW2,dfS3])
dfEWS.to_csv('data/totalpopulation_englandwalesscotland_msoaiz.csv',index=False)

In [25]:
dfPopMSOAPopulation_EWS = pd.read_csv('data/totalpopulation_englandwalesscotland_msoaiz.csv', usecols=['msoaiz', 'count_allpeople'], index_col='msoaiz') 

In [26]:
dfPopMSOAPopulation_OXF = dfPopMSOAPopulation_EWS.loc[OXF_MSOA_list]  # Filter rows
dfPopMSOAPopulation_OXF.sort_index(inplace=True)
dfPopMSOAPopulation_OXF['msoaiz'] = dfPopMSOAPopulation_OXF.index  # turn the index (i.e. MSOA codes) back into a columm
dfPopMSOAPopulation_OXF.reset_index(drop=True, inplace=True)  # IMPORTANT, otherwise indexes remain for ALL the rows i.e. idx=0..OriginalN NOT true row count!
popretailPopulation = dfPopMSOAPopulation_OXF.join(other=zonecodes_EWS.set_index('areakey'), on='msoaiz') 

In [27]:
popretailPopulation.to_csv('data/populationretailPopulation_OXF_2011.csv')

In [28]:
retailpoints_EWS = pd.read_csv('data/geolytix_retailpoints_open_regression.csv')

In [29]:
OXF_postcodes_df = pd.read_excel('data/Oxfordshire_postcodes.xlsx')  # df containing: Postcode,Latitude,Longitude,Eastings,Northings
OXF_postcodes_df['DistrictCode'] = OXF_postcodes_df['pcd'].str[:-3]  # create column with district code
OXF_DistrictCodes = OXF_postcodes_df['DistrictCode'].tolist()  # save the district codes into a list


In [30]:
open_geolitix_OXF = retailpoints_EWS.loc[retailpoints_EWS['postcode'].str[:-4].isin(OXF_DistrictCodes)]  # Filter rows (-4 because of the blank space in the middle of the postcode)
open_geolitix_OXF.reset_index(drop=True, inplace=True)

In [31]:
open_geolitix_OXF.to_csv('data/geolytix_retailpoints_open_regression_OXF.csv')

In [32]:
class QUANTLHModel:
    """
    constructor
    @param n number of residential zones (MSOA)
    @param m number of school point zones
    """

    def __init__(self, m, n):
        # constructor
        self.m = m
        self.n = n
        self.Ei = np.zeros(m)
        self.Aj = np.zeros(n)
        self.cij_0 = np.zeros(1)  # costs matrix - set to something BIG later
        self.cij_1 = np.zeros(1)  # costs matrix - set to something BIG later
        self.cij_2 = np.zeros(1)  # costs matrix - set to something BIG later
        self.SObs_0 = np.zeros(1)
        self.SObs_1 = np.zeros(1)
        self.SObs_2 = np.zeros(1)

    ################################################################################

    """
    setPopulationVectorEi
    Overload of setPopulationEi to set Ei directly from a vector, rather than a Pandas dataframe.
    """
    def setPopulationVectorEi(self, Ei):
        self.Ei = Ei
        assert len(self.Ei) == self.m, "FATAL: setPopulationEi length Ei=" + str(
            len(self.Ei)) + " MUST equal model definition size of m=" + str(self.m)

    ################################################################################

    """
    setPopulationEi
    Given a data frame containing one column with the zone number (i) and one column
    with the actual data values, fill the Ei population property with data so that
    the position in the Ei numpy array is the zonei field of the data.
    The code is almost identical to the setAttractorsAj method.
    NOTE: max(i) < self.m
    """
    def setPopulationEi(self, df, zoneiColName, dataColName):
        df2 = df.sort_values(by=[zoneiColName])
        self.Ei = df2[dataColName].to_numpy()
        assert len(self.Ei) == self.m, "FATAL: setPopulationEi length Ei=" + str(len(self.Ei)) + " MUST equal model definition size of m=" + str(self.m)

    ################################################################################

    """
    setAttractorsAj
    Given a data frame containing one column with the zone number (j) and one column
    with the actual data values, fill the Aj attractors property with data so that
    the position in the Aj numpy array is the zonej field of the data.
    The code is almost identical to the setPopulationEi method.
    NOTE: max(j) < self.n
    """
    def setAttractorsAj(self, df, zonejColName, dataColName):
        df2 = df.sort_values(by=[zonejColName])
        self.Aj = df2[dataColName].to_numpy()
        assert len(self.Aj) == self.n, "FATAL: setAttractorsAj length Aj=" + str(len(self.Aj)) + " MUST equal model definition size of n=" + str(self.n)

    ################################################################################

    """
    setCostsMatrix
    Assign the cost matrix for the model to use when it runs.
    NOTE: this MUST match the m x n order of the model and be a numpy array
    """
    def setCostMatrixCij(self, cij_0, cij_1, cij_2):
        i, j = cij_0.shape  # Hopefully cij_0.shape = cij_1.shape = cij_2.shape
        assert i == self.m, "FATAL: setCostsMatrix cij matrix is the wrong size, cij.m=" + str(i) + " MUST match model definition of m=" + str(self.m)
        assert j == self.n, "FATAL: setCostsMatrix cij matrix is the wrong size, cij.n=" + str(j) + " MUST match model definition of n=" + str(self.n)
        self.cij_0 = cij_0
        self.cij_1 = cij_1
        self.cij_2 = cij_2

    ################################################################################

    """
    setObsMatrix
    Assign the cost matrix for the model to use when it runs.
    NOTE: this MUST match the m x n order of the model and be a numpy array
    """
    def setObsMatrix(self, SObs_0, SObs_1, SObs_2):
        i, j = SObs_0.shape  # Hopefully cij_0.shape = SObs_0.shape = SObs_0.shape
        assert i == self.m, "FATAL: setCostsMatrix cij matrix is the wrong size, cij.m=" + str(
            i) + " MUST match model definition of m=" + str(self.m)
        assert j == self.n, "FATAL: setCostsMatrix cij matrix is the wrong size, cij.n=" + str(
            j) + " MUST match model definition of n=" + str(self.n)
        self.SObs_0 = SObs_0
        self.SObs_1 = SObs_1
        self.SObs_2 = SObs_2

    ################################################################################

    """
    computeCBar
    Compute average trip length TODO: VERY COMPUTATIONALLY INTENSIVE - FIX IT
    @param Sij trips matrix containing the flow numbers between MSOA (i) and schools (j)
    @param cij trip times between i and j
    """
    def computeCBar(self, Sij, cij):
        CNumerator = np.sum(Sij * cij)
        CDenominator = np.sum(Sij)
        cbar = CNumerator / CDenominator
        return cbar

    ################################################################################

    """
        Calculate Dj for a trips matrix.
        Two methods are presented here, one which is simple and very slow and one
        which uses python vector maths and is much faster. Once 2 is proven equal
        to 1, then it can be used exclusively. This function is mainly used for
        testing with the TensorFlow and other implementations.
        """

    def calculateDj(self, Tij):
        (M, N) = np.shape(Tij)
        Dj = np.zeros(N)
        Dj = Tij.sum(axis=0)
        return Dj

    ###############################################################################

    """
    run Model run3modes
    Quant model for three modes of transport
    @returns Sij predicted flows between i and j
    """
    def run3modes(self):
        # run model
        # i = employment zone
        # j = residential zone
        # Ei = number of jobs at MSOA location
        # Aj = attractor of HH (number of dwellings)
        # cij_mode = travel cost for "mode" (i.e. road, bus, rail)
        # Modes: Road = 0, Bus = 1, Rail = 2
        # Beta = Beta values for three modes - this is also output
        # QUANT data:
        # Observed trips data: "SObs_1.bin", "SObs_2.bin", "SObs_3.bin"
        # Travel cost per mode: "dis_roads_min.bin", "dis_bus_min.bin", "dis_gbrail_min.bin"
        # Note the use of 1,2,3 for modes in the files different from 0,1,2 in the code.
        # Returns predicted flows per mode: "SPred_1.bin", "SPred_2.bin", "SPred_3.bin"

        # Initialisation of parameters
        converged = False  # initialise convergence check param
        n_modes = 3  # Number of modes
        cij_k = [self.cij_0, self.cij_1, self.cij_2]  # list of cost matrices
        SObs_k = [self.SObs_0, self.SObs_1, self.SObs_2]  # list of obs trips matrices

        # Set up Beta for modes 0, 1 and 2 to 1.0
        Beta = np.ones(n_modes)

        # Compute sum of origins and destinations

        '''
        # OiObs : vector with dimension = number of oringins
        OiObs = np.zeros(self.n)
        for k in range(n_modes):
            OiObs += SObs_k[k].sum(axis=1)
        '''

        # DjObs : vector with dimension = number of destinations
        DjObs = [[] for i in range(n_modes)]
        for k in range(n_modes):
            DjObs[k] = np.zeros(self.n)
        for k in range(n_modes):
            DjObs[k] += SObs_k[k].sum(axis=1)

        DjPred = [[] for i in range(n_modes)]
        DjPredSum = np.zeros(n_modes)
        DjObsSum = np.zeros(n_modes)
        delta = np.zeros(n_modes)


        # Convergence loop:
        print("Calibrating the model...")
        iteration = 0
        while converged != True:
            iteration += 1
            print("Iteration: ", iteration)

            # Initialise variables:
            Sij = [[] for i in range(n_modes)]  # initialise Sij with a number of empty lists equal to n_modes

            # hold copy of pre multiplied copies of -Beta_k * cij[k] for each mode
            ExpMBetaCijk = [[] for k in range(n_modes)]
            for kk in range(n_modes):
                ExpMBetaCijk[kk] = np.exp(-Beta[kk] * cij_k[kk])

            for k in range(n_modes):  # mode loop
                Sij[k] = np.zeros(self.m * self.n).reshape(self.m, self.n)
                for i in range(self.m):
                    denom = 0
                    for kk in range(n_modes):
                        denom += np.sum(self.Aj * ExpMBetaCijk[kk][i, :])
                    Sij2 = self.Ei[i] * (self.Aj * ExpMBetaCijk[k][i, :] / denom)
                    Sij[k][i, :] = Sij2  # put answer slice back in return array

            # Calibration with CBar values
            # Calculate mean predicted trips and mean observed trips (this is CBar)
            CBarPred = np.zeros(n_modes)
            CBarObs = np.zeros(n_modes)
            delta = np.ones(n_modes)
            for k in range(n_modes):
                CBarPred[k] = self.computeCBar(Sij[k], cij_k[k])
                CBarObs[k] = self.computeCBar(SObs_k[k], cij_k[k])
                delta[k] = np.absolute(CBarPred[k] - CBarObs[k])  # the aim is to minimise delta[0]+delta[1]+...
            # delta check on all betas (Beta0, Beta1, Beta2) stopping condition for convergence
            # double gradient descent search on Beta0 and Beta1 and Beta2
            converged = True
            for k in range(n_modes):
                if (delta[k] / CBarObs[k] > 0.001):
                    Beta[k] = Beta[k] * CBarPred[k] / CBarObs[k]
                    converged = False
            '''
            # Calibration with Observed flows
            for k in range(n_modes):
                DjPred[k] = self.calculateDj(Sij[k])
                DjPredSum[k] = np.sum(DjPred[k])
                DjObsSum[k] = np.sum(DjObs[k])
                delta[k] = DjPredSum[k] - DjObsSum[k]
            # delta check on beta stopping condition for convergence
            # gradient descent search on beta
            converged = True
            for k in range(n_modes):
                if (delta[k] / DjObsSum[k] > 0.001):
                    Beta[k] = Beta[k] * DjPredSum[k] / DjObsSum[k]
                    converged = False
            '''
            CBarPred = np.zeros(n_modes)
            # Calculate CBar
            for k in range(n_modes):
                CBarPred[k] = self.computeCBar(Sij[k], cij_k[k])

            # Debug:
            # commuter sum blocks
            TotalSij_roads = Sij[0].sum()
            TotalSij_bus = Sij[1].sum()
            TotalSij_rail = Sij[2].sum()
            TotalEi = self.Ei.sum()  # total jobs = pu+pr above
            # print("i= {0:d} beta_roads={1:.6f} beta_bus={2:.6f} beta_rail={3:.6f} cbar_pred_roads={4:.1f} cbar_pred_busr={5:.1f} cbar_pred_rail={6:.1f}"
            #         .format(iteration, Beta[0], Beta[1], Beta[2], CBarPred[0], CBarPred[1], CBarPred[2]))
            # print("TotalSij_roads={0:.1f} TotalSij_bus={1:.1f} TotalSij_rail={2:.1f} Total={3:.1f} ({4:.1f})"
            #       .format(TotalSij_roads, TotalSij_bus, TotalSij_rail, TotalSij_roads + TotalSij_bus + TotalSij_rail, TotalEi))

        return Sij, Beta, CBarPred  # Note that Sij = [Sij_k=0 , Sij_k=1, Sij_k=2] and CBarPred = [CBarPred_0, CBarPred_1, CBarPred_2]

    ################################################################################

    """
    run Model run3modes_NoCalibration
    Quant model for three modes of transport without calibration
    @returns Sij predicted flows between i and j
    """
    def run3modes_NoCalibration(self, Beta):
        n_modes = len(Beta)  # Number of modes
        print("Running model for ", n_modes, " modes.")

        # Initialise variables:
        Sij = [[] for i in range(n_modes)]  # initialise Sij with a number of empty lists equal to n_modes
        cij_k = [self.cij_0, self.cij_1, self.cij_2]  # list of cost matrices

        # hold copy of pre multiplied copies of -Beta_k * cij[k] for each mode
        ExpMBetaCijk = [[] for k in range(n_modes)]
        for kk in range(n_modes):
            ExpMBetaCijk[kk] = np.exp(-Beta[kk] * cij_k[kk])

        for k in range(n_modes):  # mode loop
            Sij[k] = np.zeros(self.m * self.n).reshape(self.m, self.n)
            for i in range(self.m):
                denom = 0
                for kk in range(n_modes):
                    denom += np.sum(self.Aj * ExpMBetaCijk[kk][i, :])
                Sij2 = self.Ei[i] * (self.Aj * ExpMBetaCijk[k][i, :] / denom)
                Sij[k][i, :] = Sij2  # put answer slice back in return array

        CBarPred = np.zeros(n_modes)  # initialise CBarPred
        for k in range(n_modes):
            CBarPred[k] = self.computeCBar(Sij[k], cij_k[k])

        return Sij, CBarPred

    ################################################################################

    """
    computeProbabilities3modes
    Compute the probability of a flow from an MSOA zone to any (i.e. all) of the possible point zones
    """
    def computeProbabilities3modes(self, Sij):
        print("Computing probabilities")
        n_modes = 3
        probSij = [[] for i in range(n_modes)]  # initialise Sij with a number of empty list equal to n_modes

        for k in range(n_modes):
            probSij[k] = np.arange(self.m * self.n, dtype=np.float64).reshape(self.m, self.n)
            for i in range(self.m):
                sum = np.sum(Sij[k][i,])
                if sum <= 0:
                    sum = 1  # catch for divide by zero - just let the zero probs come through to the final matrix
                probSij[k][i,] = Sij[k][i,] / sum

        return probSij

In [33]:
class QUANTRetailModel(QUANTLHModel):
    """
    constructor
    @param n number of residential zones
    @param m number of retail zones
    """
    def __init__(self,m,n):
        #constructor
        super().__init__(m,n)

    ################################################################################

    """
    loadGeolytixData
    @param filename Name of file to load - this is the Geolytix restricted access data with
    the floorspace and retail data
    @returns DataFrame containing [key,zonei,east,north] and [zonei,Modelled turnover annual]
    """
    @staticmethod
    def loadGeolytixData(filename):
        missing_values = ['-', 'n/a', 'na', '--', ' -   ']
        df = pd.read_csv(filename,usecols=['id','fascia','modelled sq ft','Modelled turnover annual','bng_e','bng_n'], na_values=missing_values)
        df.dropna(axis=0,inplace=True)
        df.reset_index(drop=True,inplace=True) # IMPORTANT, otherwise indexes remain for ALL the rows i.e. idx=0..OriginalN NOT true row count!
        dfzones = pd.DataFrame({'id':df.id,'zonei':df.index,'east':df.bng_e,'north':df.bng_n})
        dfattractors = pd.DataFrame({'zonei':df.index,'Modelled turnover annual':df['Modelled turnover annual']}) # could also used floorspace
        return dfzones, dfattractors

In [34]:
popretailZones, popretailAttractors = QUANTRetailModel.loadGeolytixData('data/geolytix_retailpoints_open_regression_OXF.csv') 

In [35]:
popretailZones.to_csv('data/populationretailZones.csv')

In [36]:
popretailAttractors.to_csv('data/populationretailAttractors.csv')

In [37]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points

def costMSOAToPoint_3modes(cij_roads, cij_bus, cij_rail, dfPoints, OXF_MSOA_list):
    # const to define what speed we travel the additional distance to the retail point e.g. 30mph = 13 ms-1
    metresPerSec_roads = 13.0 # 30 miles/h = 47 km/h
    metresPerSec_bus = 13.0  # 30 miles/h = 47 km/h
    metresPerSec_rail = 13.0  # 30 miles/h = 47 km/h

    # read in the road, bus and rail centroids for the cij matrix
    df_roads = pd.read_csv(os.path.join('data', 'roadcentroidlookup_QC.csv'), index_col='zonecode')
    df_bus = pd.read_csv(os.path.join('data', 'buscentroidlookup_QC.csv'), index_col='zonecode')
    df_rail = pd.read_csv(os.path.join('data', 'gbrailcentroidlookup_QC.csv'), index_col='zonecode')

    # Filter out Oxfordshire from EWS files
    df_roads = df_roads.loc[OXF_MSOA_list]  # Filter rows
    df_bus = df_bus.loc[OXF_MSOA_list]  # Filter rows
    df_rail = df_rail.loc[OXF_MSOA_list]  # Filter rows

    df_roads['zonecode'] = df_roads.index  # turn the index (i.e. MSOA codes) back into a columm
    df_bus['zonecode'] = df_bus.index  # turn the index (i.e. MSOA codes) back into a columm
    df_rail['zonecode'] = df_rail.index  # turn the index (i.e. MSOA codes) back into a columm

    # Reset indexes
    df_roads.reset_index(drop=True, inplace=True)
    df_bus.reset_index(drop=True, inplace=True)
    df_rail.reset_index(drop=True, inplace=True)

    # Overwrite the zonei column with the new index
    df_roads['zonei'] = df_roads.index
    df_bus['zonei'] = df_bus.index
    df_rail['zonei'] = df_rail.index

    # code this into a geodataframe so we can make a spatial index
    gdf_roads = gpd.GeoDataFrame(df_roads, crs='epsg:4326', geometry=gpd.points_from_xy(df_roads.vertex_lon, df_roads.vertex_lat))
    gdf_bus = gpd.GeoDataFrame(df_bus, crs='epsg:4326', geometry=gpd.points_from_xy(df_bus.vertex_lon, df_bus.vertex_lat))
    gdf_rail = gpd.GeoDataFrame(df_rail, crs='epsg:4326', geometry=gpd.points_from_xy(df_rail.vertex_lon, df_rail.vertex_lat))

    # but it's lat/lon and we want east/north, convert crs:
    centroids_roads = gdf_roads.to_crs("EPSG:27700")
    centroids_bus = gdf_bus.to_crs("EPSG:27700")
    centroids_rail = gdf_rail.to_crs("EPSG:27700")

    dest_unary_roads = centroids_roads["geometry"].unary_union  # and need this join for the centroid points nearest lookup
    dest_unary_bus = centroids_bus["geometry"].unary_union  # and need this join for the centroid points nearest lookup
    dest_unary_rail = centroids_rail["geometry"].unary_union  # and need this join for the centroid points nearest lookup

    # create a new MSOA to points cost matix
    m, n = cij_roads.shape
    p, cols = dfPoints.shape
    # print("array size = ", p, m)

    cijpoint_roads = np.zeros(m * p, dtype=np.float64).reshape(m, p)  # so m=MSOA and p=points index
    cijpoint_bus = np.zeros(m * p, dtype=np.float64).reshape(m, p)  # so m=MSOA and p=points index
    cijpoint_rail = np.zeros(m * p, dtype=np.float64).reshape(m, p)  # so m=MSOA and p=points index

    # now make the amended cost function
    count = 0
    for row in dfPoints.itertuples(index=False):  # NOTE: iterating over Pandas rows is supposed to be bad - how else to do this?
        if (count % 50 == 0): print("costs::costMSOAToPoint ", count, "/", p)
        count += 1

        p_zonei = getattr(row, 'zonei')
        p_east = getattr(row, 'east')
        p_north = getattr(row, 'north')

        near_roads = nearest_points(Point(p_east, p_north), dest_unary_roads)
        near_bus = nearest_points(Point(p_east, p_north), dest_unary_bus)
        near_rail = nearest_points(Point(p_east, p_north), dest_unary_rail)

        match_geom_roads = centroids_roads.loc[centroids_roads.geometry == near_roads[1]]
        match_geom_bus = centroids_bus.loc[centroids_bus.geometry == near_bus[1]]
        match_geom_rail = centroids_rail.loc[centroids_rail.geometry == near_rail[1]]

        pmsoa_zonei_roads = int(match_geom_roads.zonei.iloc[0])  # closest point msoa zone
        pmsoa_zonei_bus = int(match_geom_bus.zonei.iloc[0])  # closest point msoa zone
        pmsoa_zonei_rail = int(match_geom_rail.zonei.iloc[0])  # closest point msoa zone

        pmsoa_pt_roads = match_geom_roads.geometry
        pmsoa_pt_bus = match_geom_bus.geometry
        pmsoa_pt_rail = match_geom_rail.geometry

        pmsoa_east_roads = float(pmsoa_pt_roads.centroid.x.iloc[0])
        pmsoa_east_bus = float(pmsoa_pt_bus.centroid.x.iloc[0])
        pmsoa_east_rail = float(pmsoa_pt_rail.centroid.x.iloc[0])

        pmsoa_north_roads = float(pmsoa_pt_roads.centroid.y.iloc[0])
        pmsoa_north_bus = float(pmsoa_pt_bus.centroid.y.iloc[0])
        pmsoa_north_rail = float(pmsoa_pt_rail.centroid.y.iloc[0])

        dx_roads = p_east - pmsoa_east_roads
        dx_bus = p_east - pmsoa_east_bus
        dx_rail = p_east - pmsoa_east_rail

        dy_roads = p_north - pmsoa_north_roads
        dy_bus = p_north - pmsoa_north_bus
        dy_rail = p_north - pmsoa_north_rail

        dist_roads = np.sqrt(dx_roads * dx_roads + dy_roads * dy_roads)  # dist between point and centroid used for shortest path
        dist_bus = np.sqrt(dx_bus * dx_bus + dy_bus * dy_bus)  # dist between point and centroid used for shortest path
        dist_rail = np.sqrt(dx_rail * dx_rail + dy_rail * dy_rail)  # dist between point and centroid used for shortest path

        # work out an additional delta cost based on increased time getting from this point to the centroid
        deltaCost_roads = (dist_roads / metresPerSec_roads) / 60.0  # transit time in mins
        deltaCost_bus = (dist_bus / metresPerSec_bus) / 60.0  # transit time in mins
        deltaCost_rail = (dist_rail / metresPerSec_rail) / 60.0  # transit time in mins

        # now write every cij value for msoa_zonei to p_zonei (closest) PLUS deltaCose for p_zonei to actual point
        for i in range(n):
            C1_roads = cij_roads[pmsoa_zonei_roads, i]  # yes, this is right for a trip from MSOA to closest point MSOA - QUANT is BACKWARDS
            C1_bus = cij_bus[pmsoa_zonei_bus, i]  # yes, this is right for a trip from MSOA to closest point MSOA - QUANT is BACKWARDS
            C1_rail = cij_rail[pmsoa_zonei_rail, i]  # yes, this is right for a trip from MSOA to closest point MSOA - QUANT is BACKWARDS

            cijpoint_roads[i, p_zonei] = C1_roads + deltaCost_roads
            cijpoint_bus[i, p_zonei] = C1_bus + deltaCost_bus
            cijpoint_rail[i, p_zonei] = C1_rail + deltaCost_rail

            # NOTE: you can only go in one direction with a matrix that is asymmetric

    return cijpoint_roads, cijpoint_bus, cijpoint_rail

In [38]:
retailpoints_cij_roads, retailpoints_cij_bus, retailpoints_cij_rail = costMSOAToPoint_3modes(cij_road_OXF, cij_bus_OXF, cij_rail_OXF, popretailZones, OXF_MSOA_list)

costs::costMSOAToPoint  0 / 520
costs::costMSOAToPoint  50 / 520
costs::costMSOAToPoint  100 / 520
costs::costMSOAToPoint  150 / 520
costs::costMSOAToPoint  200 / 520
costs::costMSOAToPoint  250 / 520
costs::costMSOAToPoint  300 / 520
costs::costMSOAToPoint  350 / 520
costs::costMSOAToPoint  400 / 520
costs::costMSOAToPoint  450 / 520
costs::costMSOAToPoint  500 / 520


In [39]:
saveMatrix(retailpoints_cij_roads, 'data/retailpointsCij_roads.bin')
saveMatrix(retailpoints_cij_bus, 'data/retailpointsCij_bus.bin')
saveMatrix(retailpoints_cij_rail, 'data/retailpointsCij_rail.bin')

In [40]:
np.savetxt('data/retailpointsCij_roads.csv', retailpoints_cij_roads, delimiter=",")
np.savetxt('data/retailpointsCij_bus.csv', retailpoints_cij_bus, delimiter=",")
np.savetxt('data/retailpointsCij_rail.csv', retailpoints_cij_rail, delimiter=",")

In [41]:
m, n = retailpoints_cij_roads.shape
model = QUANTRetailModel(m,n)

In [42]:
model.setAttractorsAj(popretailAttractors,'zonei','Modelled turnover annual')
model.setPopulationEi(popretailPopulation,'zonei','count_allpeople')
model.setCostMatrixCij(retailpoints_cij_roads, retailpoints_cij_bus, retailpoints_cij_rail)

In [43]:
def run3modes_NoCalibration(self, Beta):
        n_modes = len(Beta)  # Number of modes
        print("Running model for ", n_modes, " modes.")

        # Initialise variables:
        Sij = [[] for i in range(n_modes)]  # initialise Sij with a number of empty lists equal to n_modes
        cij_k = [self.cij_0, self.cij_1, self.cij_2]  # list of cost matrices

        # hold copy of pre multiplied copies of -Beta_k * cij[k] for each mode
        ExpMBetaCijk = [[] for k in range(n_modes)]
        for kk in range(n_modes):
            ExpMBetaCijk[kk] = np.exp(-Beta[kk] * cij_k[kk])

        for k in range(n_modes):  # mode loop
            Sij[k] = np.zeros(self.m * self.n).reshape(self.m, self.n)
            for i in range(self.m):
                denom = 0
                for kk in range(n_modes):
                    denom += np.sum(self.Aj * ExpMBetaCijk[kk][i, :])
                Sij2 = self.Ei[i] * (self.Aj * ExpMBetaCijk[k][i, :] / denom)
                Sij[k][i, :] = Sij2  # put answer slice back in return array

        CBarPred = np.zeros(n_modes)  # initialise CBarPred
        for k in range(n_modes):
            CBarPred[k] = self.computeCBar(Sij[k], cij_k[k])

        return Sij, CBarPred

In [44]:
# load jobs data for MSOA residential zones
dfEi = pd.read_csv('data/Employment_by_sector_GB_MSOA.csv', usecols=['geography code','Industry: All categories: Industry; measures: Value'], index_col='geography code')

In [45]:
dfEi.rename(columns={'geography code': 'msoa', 'Industry: All categories: Industry; measures: Value': 'employment_tot'}, inplace=True)

    # Filter out Oxfordshire from EW dataset:
dfEi = dfEi.loc[OXF_MSOA_list]  # Filter rows

dfEi['msoa'] = dfEi.index # turn the index (i.e. MSOA codes) back into a columm

dfEi.reset_index(drop=True, inplace=True)  # IMPORTANT, otherwise indexes remain for ALL the rows i.e. idx=0..OriginalN NOT true row count!

    # drop columns:
dfEi = dfEi[['msoa','employment_tot']]
jobs = dfEi.join(other=zonecodes_EWS.set_index('areakey'), on='msoa')

In [46]:
data_census_QS103_EW_df = pd.read_csv('data/QS103EW_MSOA.csv', index_col='geography code')
data_census_QS103_OXF_df = data_census_QS103_EW_df.loc[OXF_MSOA_list]  # Filter rows
data_census_QS103_OXF_df['geography code'] = data_census_QS103_OXF_df.index # turn the index (i.e. MSOA codes) back into a columm
data_census_QS103_OXF_df.reset_index(drop=True, inplace=True)
data_census_QS103_OXF_df.to_csv('data/QS103EW_MSOA_OXF.csv')

In [47]:
"""
utils.py
Data building utilities
"""

import numpy as np
import pickle
import struct
import csv

###############################################################################
"""
Load the zone codes lookup from a csv file into a dictionary of dictionaries
"""
#MOVED TO zonecodes.py
# def loadZoneLookup(filename):
#     ZoneLookup = {}
#     with open(filename,'r') as csv_file:
#         reader = csv.reader(csv_file, delimiter=',')
#         header = next(reader,None) #skip header line
#         for row in reader:
#             #zonei,areakey,name,east,north
#             #0,E02000001,City of London 001,532482.7,181269.3
#             zonei = int(row[0])
#             msoa = row[1]
#             name = row[2]
#             east = float(row[3])
#             north = float(row[4])
#             ZoneLookup[msoa] = {
#                 'zonei': zonei,
#                 'name': name,
#                 'east': east,
#                 'north': north
#             }
#             #print("loadZoneLookup: ",msoa,zonei,name,east,north) #DEBUG
#     return ZoneLookup
###############################################################################

"""
Build a trips matrix from data in the Census CSV file (i.e. wu03ew_v2.csv).
The point behind this is to be able to build separate matrices from the sums of a user defined set of columns.
This does assume that the two area keys for the origin destination areas are the first two columns in the file.
NOTE: this defines which way round the i and j are. See comment in code main loop below.
@param name="CSVFilename" string
@param name="ZoneLookup" Lookup between MSOA area key and ZoneCode index number Dictionary of dictionaries
@param name="ColumnNames" Names of columns added together to get the total trips i.e. "UNDERGROUND", "TRAIN", "BUS". These must be in the CSV header line List of string
@returns An observed trips matrix (numpy) matrix
NOTE: you will get a LOT of warnings from this - as long as they're for ODxxx or S92xxx zones, then this is normal
"""
def generateTripsMatrix(CSVFilename, ZoneLookup, ColumnNames):
    N = len(ZoneLookup)
    TijObs = np.zeros(N*N).reshape(N, N)
    for i in range(0,N): # this is basically a hack to absolutely ensure everything is zeroed - above arange sets it to col and row values?
        for j in range(0,N):
            TijObs[i,j]=0.0
    print(ColumnNames,len(ColumnNames))

    with open(CSVFilename) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        fields = next(reader,None)
        # work out column index numbers for all the column names
        ColI = [-1 for i in range(0,len(ColumnNames))] #init with -1
        for i in range(len(fields)):
            field = fields[i]
            for j in range(0,len(ColumnNames)):
                if field == ColumnNames[j]:
                    ColI[j] = i
        print(ColumnNames,ColI)

        # "Area of residence","Area of workplace","All categories: Method of travel to work","Work mainly at or from home","Underground, metro, light rail, tram","Train","Bus, minibus or coach","Taxi","Motorcycle, scooter or moped","Driving a car or van","Passenger in a car or van","Bicycle","On foot","Other method of travel to work"
        # E02000001,E02000001,1506,0,73,41,32,9,1,8,1,33,1304,4
        lineCount = 1
        for row in reader:
            lineCount+=1
            ZoneR = row[0]
            ZoneW = row[1]
            sum = 0
            for i in range(len(ColI)):
                count = int(row[ColI[i]])
                sum += count
            RowR = ZoneLookup.get(ZoneR,'Empty') # could potentially fail if ZoneR or ZoneW didn't exist in the shapefile
            RowW = ZoneLookup.get(ZoneW,'Empty')
            if RowR == 'Empty' or RowW == 'Empty':
                print("Warning: trip " + ZoneR + " to " + ZoneW + " zones not found - skipped")
                continue
            ZoneR_idx = RowR["zonei"]
            ZoneW_idx = RowW["zonei"]
            # TijObs[ZoneR_idx, ZoneW_idx] = sum #this was the original that was apparently the wrong way around
            TijObs[ZoneW_idx, ZoneR_idx] = sum #this is the above line with i and j flipped - right way around
        print('Loaded ', CSVFilename, ' and processed ', lineCount, ' lines of data')
        print('Finished GenerateTripsMatrix')

    return TijObs

###############################################################################

"""
Load a numpy matrix from a file
"""
def loadMatrix(filename):
    with open(filename,'rb') as f:
        matrix = pickle.load(f)
    return matrix

###############################################################################

"""
Save a numpy matrix to a file
"""
def saveMatrix(matrix,filename):
    with open(filename,'wb') as f:
        pickle.dump(matrix,f)

###############################################################################

"""
Load a QUANT format matrix into python.
A QUANT matrix stores the row count (m), column count (n) and then m x n IEEE754 floats (4 byte) of data
"""
def loadQUANTMatrix(filename):
    with open(filename,'rb') as f:
        (m,) = struct.unpack('i', f.read(4))
        (n,) = struct.unpack('i', f.read(4))
        print("loadQUANTMatrix::m=",m,"n=",n)
        matrix = np.arange(m*n,dtype=np.float).reshape(m, n) #and hopefully m===n, but I'm not relying on it
        for i in range(0,m):
            data = struct.unpack('{0}f'.format(n), f.read(4*n)) #read a row back from the stream - data=list of floats
            for j in range(0,n):
                matrix[i,j] = data[j]
    return matrix

###############################################################################

"""
Load CSV data from QUANT in the form of i,j,Oi,Dj,Cij,Tij
The N parameter specifies the matrix size i.e. 7201. Technically we could read
this from the csv if it was complete, but it's easier to pass in
NOTE: the csv data is already in the right format to make training data from, but
this puts it back into a Tij and Cij matrix that we use to build it from again
"""
def loadQUANTCSV(filename,N):
    Tij = np.zeros(N*N).reshape(N,N)
    Cij = np.zeros(N*N).reshape(N,N)
    with open(filename,'r') as f:
        lines=f.readlines() # read everything in to memory at once - it's a lot faster, but there are 52M
        for n in range(1,len(lines)): # need to skip the header
            # so it's i,j,Oi,Dj,Cij,Tij
            fields = lines[n].split(',')
            if len(fields)==6:
                i = int(fields[0])
                j = int(fields[1])
                # Oi = float(fields[2])
                # Dj = float(fields[3])
                valCij = float(fields[4])
                valTij = float(fields[5]) # or int?
                Tij[i,j]=valTij
                Cij[i,j]=valCij
    return Cij, Tij

###############################################################################

"""
Resize a matrix: if smaller then rows and cols are cut out, if bigger then rows and cols are repeated modulo
This is used for the benchmarks, so I can vary the matrix size and see how long it takes
"""
def resizeMatrix(matrix,N):
    (m, n) = np.shape(matrix) # original matrix size
    m2 = np.zeros(N*N).reshape(N, N)
    for i in range(0,N):
        for j in range(0,N):
            m2[i,j]=matrix[i % n,j % n]
    return m2

In [48]:
class QUANTJobsModel(QUANTLHModel):
    """
    constructor
    @param n number of residential zones
    @param m number of retail zones
    """

    def __init__(self, m, n):
        # constructor
        super().__init__(m, n)

    """
    loadEmploymentData
    - Load jobs by zone: df with columns = [,zonei,employment]
    - Load floorspace by zone: df with columns = [,zonei,floorspace] 
    return dfzones, dfattractors (they are DataFrames)
    """
    @staticmethod
    def loadEmploymentData_HHAttractiveness(filename_pop, filename_attractiveness, Scenario):
        missing_values = ['-', 'n/a', 'na', '--', ' -   ']

        # load population data for MSOA residential zones
        df = pd.read_csv(filename_pop,
                         usecols=['geography code', 'Age: All categories: Age; measures: Value'],
                         na_values=missing_values)  # import population data for England and Wales

        df.reset_index(drop=True, inplace=True)  # IMPORTANT, otherwise indexes remain for ALL the rows i.e. idx=0..OriginalN NOT true row count!

        # Rename columns:
        df.rename(columns={'geography code': 'zonei',
                           'Age: All categories: Age; measures: Value': 'Population_tot'}, inplace=True)

        if Scenario == '2011':
            df2 = pd.read_csv(filename_attractiveness, encoding='latin-1') # encoding added to solve: UnicodeDecodeError
        elif Scenario == 'NewHousingDev_2019':
            df2 = pd.read_csv(filename_attractiveness, encoding='latin-1') # encoding added to solve: UnicodeDecodeError
        elif Scenario == 'NewHousingDev_2030':
            df2 = pd.read_csv(filename_attractiveness, encoding='latin-1') # encoding added to solve: UnicodeDecodeError

        df2.rename(columns={'MSOA_Code': 'zonei', 'N_of_Dwellings': 'N_of_Dwellings'}, inplace=True)

        # Convert N_of_Dwellings to float
        df2['N_of_Dwellings'] = df2.N_of_Dwellings.astype(float)

        dfzones = pd.DataFrame({'zonei': df.zonei, 'Population_tot': df.Population_tot})
        dfattractors = pd.DataFrame({'zonei': df2.zonei, 'N_of_Dwellings': df2.N_of_Dwellings})
        return dfzones, dfattractors

    @staticmethod
    def loadEmploymentData_JobAttractiveness(filename_jobs, filename_attractiveness): # Attractiveness: offices floorspace
        missing_values = ['-', 'n/a', 'na', '--', ' -   ']

        df = pd.read_csv(filename_jobs,
                         usecols=['geography code','Industry: All categories: Industry; measures: Value'], # select the columms that I need from file
                         na_values=missing_values)
        #df.dropna(axis=0, inplace=True) # I think I want to keep all the MSOAs even if they have null alues - in case change them

        df.reset_index(drop=True, inplace=True)  # IMPORTANT, otherwise indexes remain for ALL the rows i.e. idx=0..OriginalN NOT true row count!

        # Rename columns:
        df.rename(columns={'geography code': 'zonei', 'Industry: All categories: Industry; measures: Value': 'employment_tot'}, inplace=True)

        df2 = pd.read_csv(filename_attractiveness,
                          encoding='latin-1', # added to solve: UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf4 in position 25: invalid continuation byte
                         usecols=['Geography', 'area_code', 'Area_Name', 'Floorspace_2018-19_Total'])  # select the columms that I need from file
                         #na_values=missing_values)
        #df2.dropna(axis=0, inplace=True) # I think I want to keep all the MSOAs even if they have null alues - in case change them

        # filter out MSOA rows in jobs_floorspace:
        df2 = df2[df2.Geography == 'MSOA']

        df2.reset_index(drop=True, inplace=True)  # IMPORTANT, otherwise indexes remain for ALL the rows i.e. idx=0..OriginalN NOT true row count!

        df2.rename(columns={'area_code': 'zonei', 'Floorspace_2018-19_Total': 'floorspace'}, inplace=True)

        for m in missing_values:
            df2['floorspace'] = df2.floorspace.str.replace(m,'1.0') # Replace missing values with the minimum value of df2: 1.0

        # Convert floorspace to float
        df2['floorspace'] = df2.floorspace.str.replace(',', '').astype(float)

        #df2.fillna(1.0)  # Replace nan with the minimum value of df2 - 1.0

        dfzones = pd.DataFrame({'zonei': df.zonei, 'employment_tot': df.employment_tot})
        dfattractors = pd.DataFrame({'zonei': df2.zonei, 'floorspace': df2.floorspace})
        return dfzones, dfattractors

    @staticmethod
    def loadObsData():
        # load observed trips for each mode:
        SObs_0 = loadMatrix(os.path.join('data', 'SObs_1_OXF.bin'))
        SObs_1 = loadMatrix(os.path.join('data', 'SObs_2_OXF.bin'))
        SObs_2 = loadMatrix(os.path.join('data', 'SObs_3_OXF.bin'))

        return SObs_0, SObs_1, SObs_2

In [49]:
HHZones, HHAttractors = QUANTJobsModel.loadEmploymentData_HHAttractiveness('data/QS103EW_MSOA_OXF.csv', 'data/OXF-Dwellings2011.csv', '2011')

In [50]:
HHZones.to_csv('data/jobs_Pop_Zones_2011.csv')

In [51]:
HHAttractors.to_csv('data/jobs_HH_Attractors_2011.csv')

In [52]:
SObs_road, SObs_bus, SObs_rail = QUANTJobsModel.loadObsData()

In [53]:
# Use cij as cost matrix (MSOA to MSOA)
m, n = cij_road_OXF.shape
model = QUANTJobsModel(m, n)
model.setObsMatrix(SObs_road, SObs_bus, SObs_rail)
model.setAttractorsAj(HHAttractors, 'zonei', 'N_of_Dwellings')
model.setPopulationEi(jobs, 'zonei', 'employment_tot')
model.setCostMatrixCij(cij_road_OXF, cij_bus_OXF, cij_rail_OXF)

In [54]:
def run3modes(self):
        # run model
        # i = employment zone
        # j = residential zone
        # Ei = number of jobs at MSOA location
        # Aj = attractor of HH (number of dwellings)
        # cij_mode = travel cost for "mode" (i.e. road, bus, rail)
        # Modes: Road = 0, Bus = 1, Rail = 2
        # Beta = Beta values for three modes - this is also output
        # QUANT data:
        # Observed trips data: "SObs_1.bin", "SObs_2.bin", "SObs_3.bin"
        # Travel cost per mode: "dis_roads_min.bin", "dis_bus_min.bin", "dis_gbrail_min.bin"
        # Note the use of 1,2,3 for modes in the files different from 0,1,2 in the code.
        # Returns predicted flows per mode: "SPred_1.bin", "SPred_2.bin", "SPred_3.bin"

        # Initialisation of parameters
        converged = False  # initialise convergence check param
        n_modes = 3  # Number of modes
        cij_k = [self.cij_0, self.cij_1, self.cij_2]  # list of cost matrices
        SObs_k = [self.SObs_0, self.SObs_1, self.SObs_2]  # list of obs trips matrices

        # Set up Beta for modes 0, 1 and 2 to 1.0
        Beta = np.ones(n_modes)

        # Compute sum of origins and destinations

        '''
        # OiObs : vector with dimension = number of oringins
        OiObs = np.zeros(self.n)
        for k in range(n_modes):
            OiObs += SObs_k[k].sum(axis=1)
        '''

        # DjObs : vector with dimension = number of destinations
        DjObs = [[] for i in range(n_modes)]
        for k in range(n_modes):
            DjObs[k] = np.zeros(self.n)
        for k in range(n_modes):
            DjObs[k] += SObs_k[k].sum(axis=1)

        DjPred = [[] for i in range(n_modes)]
        DjPredSum = np.zeros(n_modes)
        DjObsSum = np.zeros(n_modes)
        delta = np.zeros(n_modes)


        # Convergence loop:
        print("Calibrating the model...")
        iteration = 0
        while converged != True:
            iteration += 1
            print("Iteration: ", iteration)

            # Initialise variables:
            Sij = [[] for i in range(n_modes)]  # initialise Sij with a number of empty lists equal to n_modes

            # hold copy of pre multiplied copies of -Beta_k * cij[k] for each mode
            ExpMBetaCijk = [[] for k in range(n_modes)]
            for kk in range(n_modes):
                ExpMBetaCijk[kk] = np.exp(-Beta[kk] * cij_k[kk])

            for k in range(n_modes):  # mode loop
                Sij[k] = np.zeros(self.m * self.n).reshape(self.m, self.n)
                for i in range(self.m):
                    denom = 0
                    for kk in range(n_modes):
                        denom += np.sum(self.Aj * ExpMBetaCijk[kk][i, :])
                    Sij2 = self.Ei[i] * (self.Aj * ExpMBetaCijk[k][i, :] / denom)
                    Sij[k][i, :] = Sij2  # put answer slice back in return array

            # Calibration with CBar values
            # Calculate mean predicted trips and mean observed trips (this is CBar)
            CBarPred = np.zeros(n_modes)
            CBarObs = np.zeros(n_modes)
            delta = np.ones(n_modes)
            for k in range(n_modes):
                CBarPred[k] = self.computeCBar(Sij[k], cij_k[k])
                CBarObs[k] = self.computeCBar(SObs_k[k], cij_k[k])
                delta[k] = np.absolute(CBarPred[k] - CBarObs[k])  # the aim is to minimise delta[0]+delta[1]+...
            # delta check on all betas (Beta0, Beta1, Beta2) stopping condition for convergence
            # double gradient descent search on Beta0 and Beta1 and Beta2
            converged = True
            for k in range(n_modes):
                if (delta[k] / CBarObs[k] > 0.001):
                    Beta[k] = Beta[k] * CBarPred[k] / CBarObs[k]
                    converged = False
            '''
            # Calibration with Observed flows
            for k in range(n_modes):
                DjPred[k] = self.calculateDj(Sij[k])
                DjPredSum[k] = np.sum(DjPred[k])
                DjObsSum[k] = np.sum(DjObs[k])
                delta[k] = DjPredSum[k] - DjObsSum[k]
            # delta check on beta stopping condition for convergence
            # gradient descent search on beta
            converged = True
            for k in range(n_modes):
                if (delta[k] / DjObsSum[k] > 0.001):
                    Beta[k] = Beta[k] * DjPredSum[k] / DjObsSum[k]
                    converged = False
            '''
            CBarPred = np.zeros(n_modes)
            # Calculate CBar
            for k in range(n_modes):
                CBarPred[k] = self.computeCBar(Sij[k], cij_k[k])

            # Debug:
            # commuter sum blocks
            TotalSij_roads = Sij[0].sum()
            TotalSij_bus = Sij[1].sum()
            TotalSij_rail = Sij[2].sum()
            TotalEi = self.Ei.sum()  # total jobs = pu+pr above
            # print("i= {0:d} beta_roads={1:.6f} beta_bus={2:.6f} beta_rail={3:.6f} cbar_pred_roads={4:.1f} cbar_pred_busr={5:.1f} cbar_pred_rail={6:.1f}"
            #         .format(iteration, Beta[0], Beta[1], Beta[2], CBarPred[0], CBarPred[1], CBarPred[2]))
            # print("TotalSij_roads={0:.1f} TotalSij_bus={1:.1f} TotalSij_rail={2:.1f} Total={3:.1f} ({4:.1f})"
            #       .format(TotalSij_roads, TotalSij_bus, TotalSij_rail, TotalSij_roads + TotalSij_bus + TotalSij_rail, TotalEi))

        return Sij, Beta, CBarPred 

In [55]:
Tij, beta_k, cbar_k = model.run3modes()

Calibrating the model...
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14


In [74]:
beta_k

array([0.3127422 , 0.07877828, 0.05523665])

In [56]:
beta_k

array([0.3127422 , 0.07877828, 0.05523665])

In [57]:
def run3modes_NoCalibration(self, Beta):
        n_modes = len(Beta)  # Number of modes
        print("Running model for ", n_modes, " modes.")

        # Initialise variables:
        Sij = [[] for i in range(n_modes)]  # initialise Sij with a number of empty lists equal to n_modes
        cij_k = [self.cij_0, self.cij_1, self.cij_2]  # list of cost matrices

        # hold copy of pre multiplied copies of -Beta_k * cij[k] for each mode
        ExpMBetaCijk = [[] for k in range(n_modes)]
        for kk in range(n_modes):
            ExpMBetaCijk[kk] = np.exp(-Beta[kk] * cij_k[kk])

        for k in range(n_modes):  # mode loop
            Sij[k] = np.zeros(self.m * self.n).reshape(self.m, self.n)
            for i in range(self.m):
                denom = 0
                for kk in range(n_modes):
                    denom += np.sum(self.Aj * ExpMBetaCijk[kk][i, :])
                Sij2 = self.Ei[i] * (self.Aj * ExpMBetaCijk[k][i, :] / denom)
                Sij[k][i, :] = Sij2  # put answer slice back in return array

        CBarPred = np.zeros(n_modes)  # initialise CBarPred
        for k in range(n_modes):
            CBarPred[k] = self.computeCBar(Sij[k], cij_k[k])

        return Sij, CBarPred

In [58]:
Rij, cbar = model.run3modes_NoCalibration(beta_k)

Running model for  3  modes.


In [59]:
def computeProbabilities3modes(self, Sij):
        print("Computing probabilities")
        n_modes = 3
        probSij = [[] for i in range(n_modes)]  # initialise Sij with a number of empty list equal to n_modes

        for k in range(n_modes):
            probSij[k] = np.arange(self.m * self.n, dtype=np.float).reshape(self.m, self.n)
            for i in range(self.m):
                sum = np.sum(Sij[k][i,])
                if sum <= 0:
                    sum = 1  # catch for divide by zero - just let the zero probs come through to the final matrix
                probSij[k][i,] = Sij[k][i,] / sum

        return probSij

In [60]:
# Compute Probabilities:
popretail_probRij = model.computeProbabilities3modes(Rij)

Computing probabilities


In [61]:
np.savetxt('data/populationretailProbRij_roads_2011.csv', popretail_probRij[0], delimiter=",")
np.savetxt('data/populationretailProbRij_bus_2011.csv', popretail_probRij[1], delimiter=",")
np.savetxt('data/populationretailProbRij_rail_2011.csv', popretail_probRij[2], delimiter=",")

In [62]:
np.savetxt('data/populationretailRij_roads_2011.csv', Rij[0], delimiter=",")
np.savetxt('data/populationretailRij_bus_2011.csv', Rij[1], delimiter=",")
np.savetxt('data/populationretailRij_rail_2011.csv', Rij[2], delimiter=",")

In [63]:
import geopandas as gpd

gdf = gpd.read_file('data/ESRI/MSOA_2011_London_gen_MHW.shp')

In [64]:
gdf = gdf.to_crs('EPSG: 4326')

In [65]:
gdf['geometry'] = gdf['geometry'].centroid

gdf = gdf[['geometry']]

/var/folders/63/l2ny8d3j1rl_k0h8ghjdkflm0000gn/T/ipykernel_3470/1845079915.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf['geometry'].centroid


In [66]:
gdf.to_file('data/OXF_MSOA_centroids_WGS84.shp')

In [67]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from matplotlib_scalebar.scalebar import ScaleBar
import networkx as nx
import os
import osmnx as ox

def calc_shortest_paths_ODs_osm(zones_centroids, network):
    # For each zone centroid, this function calculates the closest node in the OSM graph.
    # These nodes will be used as origins and destinations in the shortest paths calculations.
    list_of_ODs = []
    for c in zones_centroids:
        graph_clostest_node = ox.nearest_nodes(network, c[0], c[1], return_dist=False)
        list_of_ODs.append(graph_clostest_node)
    return list_of_ODs

def flows_map_creation(inputs, outputs, flows_output_keys): # Using OSM

    Zone_nodes = nx.read_shp('data/OXF_MSOA_centroids_WGS84.shp') # Must be in epsg:4326 (WGS84)


    Case_Study_Zones = ['London']
    # Case_Study_Zones = ['Oxford'] # Smaller network for test purposes

    X = ox.graph_from_place(Case_Study_Zones, network_type='drive')
    # crs = X.graph["crs"]
    # print('Graph CRS: ', crs)
    # print()

    # ox.plot_graph(X) # test plot

    X = X.to_undirected()

    # Calculate the origins and destinations for the shortest paths algorithms to be run on OSM graph
    OD_list = calc_shortest_paths_ODs_osm(Zone_nodes, X)

    Flows = []

    for kk, flows_output_key in enumerate(flows_output_keys):
        Flows.append(pd.read_csv(outputs[flows_output_key], header=None))

        # Initialise weights to 0:
        for source, target in X.edges():
            X[source][target][0]["Flows_" + str(kk)] = 0

    TOT_count = len(OD_list)
    # print(OD_list)

    for n, i in enumerate(OD_list):
        print("Flows maps creation - iteration ", n+1, " of ", TOT_count)
        sssp_paths = nx.single_source_dijkstra_path(X, i, weight='length') # single source shortest paths from i to all nodes of the network
        for m, j in enumerate(OD_list):
            shortest_path = sssp_paths[j] # shortest path from i to j
            path_edges = zip(shortest_path, shortest_path[1:])  # Create edges from nodes of the shortest path

            for edge in list(path_edges):
                for cc in range(len(Flows)):
                    X[edge[0]][edge[1]][0]["Flows_" + str(cc)] += Flows[cc].iloc[n, m]

    # save graph to shapefile
    output_folder_path = "data" + "Flows_shp"
    ox.save_graph_shapefile(X, filepath=output_folder_path)

In [68]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from matplotlib_scalebar.scalebar import ScaleBar
import networkx as nx
import os
import osmnx as ox

In [69]:
import sys
print(sys.executable)

/Users/pinlyu/opt/anaconda3/bin/python


In [70]:
import geopandas as gpd
import shapefile as shp
import osmnx as ox
import networkx as nx
import pandas as pd
from shapely.geometry import Point

def calc_shortest_paths_ODs_osm(zones_centroids, network):
    list_of_ODs = []
    for c in zones_centroids.geometry:
        graph_clostest_node = ox.nearest_nodes(network, c.x, c.y, return_dist=False)
        list_of_ODs.append(graph_clostest_node)
    return list_of_ODs

def flows_map_creation(outputs, flows_output_keys):
    # Read shapefile using GeoPandas
    Zone_nodes = gpd.read_file('data/OXF_MSOA_centroids_WGS84.shp')

    Case_Study_Zones = ['London']

    X = ox.graph_from_place(Case_Study_Zones, network_type='drive')
    X = X.to_undirected()

    # Convert GeoDataFrame to suitable format for calc_shortest_paths_ODs_osm function
    OD_list = calc_shortest_paths_ODs_osm(Zone_nodes, X)

    Flows = []

    for kk, flows_output_key in enumerate(flows_output_keys):
        Flows.append(pd.read_csv(outputs[flows_output_key], header=None))

        # Initialise weights to 0:
        for source, target in X.edges():
            X[source][target][0]["Flows_" + str(kk)] = 0

    TOT_count = len(OD_list)

    for n, i in enumerate(OD_list):
        print("Flows maps creation - iteration ", n+1, " of ", TOT_count)
        sssp_paths = nx.single_source_dijkstra_path(X, i, weight='length') 
        for m, j in enumerate(OD_list):
            shortest_path = sssp_paths[j] 
            path_edges = zip(shortest_path, shortest_path[1:])  

            for edge in list(path_edges):
                for cc in range(len(Flows)):
                    X[edge[0]][edge[1]][0]["Flows_" + str(cc)] += Flows[cc].iloc[n, m]

    # save graph to shapefile using PyShp
    w = shp.Writer('data/Flows_shp')
    w.field('ID')
    id = 0
    for edge in X.edges(data=True):
        w.line([[[edge[0][1], edge[0][0]], [edge[1][1], edge[1][0]]]])
        w.record(id)
        id += 1
    w.close()